In [1]:
import random

import catboost as cb
import tqdm

import utils

%load_ext autoreload
%autoreload 2

## Готовим данные для анализа и тренировки

### По этим путям лежит выход Code Completion Evaluation Plugin

In [2]:
paths = [
    "/Users/ifled/projects/csc/ide_perf_practice/detekt/completion-evaluation/2022-05-18_16-07-33",
    "/Users/ifled/projects/csc/ide_perf_practice/detekt/completion-evaluation/2022-05-02_16-55-43"
]

### Считываем признаки, которые используются при ранжировании

In [3]:
%%time

id2feats = utils.read_id2feats(paths)

Read 100 feats
Read 200 feats
Read 300 feats
Read 400 feats
Read 500 feats
Read 600 feats
Read 700 feats
Read 800 feats
Read 900 feats
Read 1000 feats
Read 1100 feats
Read 1200 feats
Read 1300 feats
Read 1400 feats
Read 1500 feats
Read 1600 feats
Read 1700 feats
Read 1800 feats
Read 1900 feats
Read 2000 feats
Read 2100 feats
Read 2200 feats
Read 2300 feats
Read 2400 feats
Read 2500 feats
Read 2600 feats
Read 2700 feats
Read 2800 feats
Read 2900 feats
Read 3000 feats
Read 3100 feats
Read 3200 feats
Read 3300 feats
Read 3400 feats
Read 3500 feats


CPU times: user 29.9 s, sys: 2.2 s, total: 32.1 s
Wall time: 39.9 s


Read 3563 feats


In [4]:
len(id2feats)

3563

In [5]:
list(id2feats.keys())[:10]

['f6e90cc2-27ac-4c99-867e-566871074918',
 '3b6adb85-a76d-469d-82b1-6c54fd9ef373',
 '0aa5cd60-0142-4d11-8b8f-c2e508eb3613',
 'fd741f4b-c011-4515-b2e3-c56a3af102a2',
 '178b3f9f-5ca1-4349-9e94-88df79083635',
 '882310c1-90a8-4329-bcde-c684a8012e2d',
 '6c5510db-79c7-442c-97eb-d97852c3daf5',
 '99148856-1ae0-4913-b232-92749d8ce858',
 '966917c4-7662-4f2c-9520-f009928f0cec',
 '18b8aecf-1957-4b50-b591-54ad82ed99ab']

### Признаки для одной сессии выглядят так (использоваться будут только context):

In [6]:
id2feats["fd741f4b-c011-4515-b2e3-c56a3af102a2"]

[{'common': {'context': {'ml_ctx_kotlin_plugin_version': '@snapshot@',
    'ml_ctx_kotlin_file_type': 'KT',
    'ml_ctx_cce_sessionUid': 'fd741f4b-c011-4515-b2e3-c56a3af102a2',
    'ml_ctx_common_line_num': '15.0',
    'ml_ctx_common_col_num': '17.0',
    'ml_ctx_common_indent_level': '2.0',
    'ml_ctx_common_is_in_line_beginning': '0',
    'ml_ctx_common_case_sensitivity': 'FIRST_LETTER',
    'ml_ctx_common_is_after_dot': '0',
    'ml_ctx_common_parent_1': 'KtPropertyAccessor',
    'ml_ctx_common_parent_2': 'KtProperty',
    'ml_ctx_common_parent_3': 'KtClassBody',
    'ml_ctx_common_parent_4': 'KtClass',
    'ml_ctx_common_parent_5': 'KtFile'},
   'user': {'completionFinishTypeTypedSelect:App': '0.05737704918032787',
    'completionFinishTypeTypedSelect:Project': '0.05737704918032787',
    'completionFinishTypeExplicitSelect:App': '0.8852459016393442',
    'completionFinishTypeExplicitSelect:Project': '0.8852459016393442',
    'completionFinishTypeCancelled:App': '0.0573770491803278

### Считываем список элементов, текст в них, тайминги и имя контрибьютора и части контрибьютора

### Ну и словарь, который для каждой сессии говорит, к какому файлу она относится (словарь используется для разбиения на train/dev/test)

In [7]:
%%time

id2session, id2fname = utils.read_sessions(paths)

Read 100 sessions
Read 200 sessions
Read 300 sessions
Read 400 sessions
Read 500 sessions
Read 600 sessions
Read 700 sessions
Read 800 sessions
Read 900 sessions
Read 1000 sessions
Read 1100 sessions
Read 1200 sessions
Read 1300 sessions
Read 1400 sessions
Read 1500 sessions
Read 1600 sessions
Read 1700 sessions
Read 1800 sessions
Read 1900 sessions
Read 2000 sessions
Read 2100 sessions
Read 2200 sessions
Read 2300 sessions
Read 2400 sessions
Read 2500 sessions
Read 2600 sessions
Read 2700 sessions
Read 2800 sessions
Read 2900 sessions
Read 3000 sessions
Read 3100 sessions
Read 3200 sessions
Read 3300 sessions


CPU times: user 5.49 s, sys: 736 ms, total: 6.23 s
Wall time: 6.4 s


Read 3400 sessions
Read 3500 sessions
Read 3570 sessions


In [8]:
len(id2fname)

3570

### Для семи сессий признаки не посчитались

In [9]:
list(id2fname.keys())[:10]

['1a230fb5-e045-42fe-8716-089e2027744e',
 '58d81941-e5cb-44ae-b7b7-94169497d048',
 'cfb26963-0190-4fb0-9556-255949566a06',
 '555e1081-987b-4e33-a272-514c4b579292',
 'c1f62c29-fbf9-4aa9-85d8-9e41c9c261e0',
 '83b00da6-e2c3-4e64-8f74-4b7f90fefe46',
 '852e57b7-cc18-45ae-af08-1f4c23c65900',
 'e0dde575-a909-4687-be08-d1b7399c9602',
 '22cc3e61-42a4-47f4-85ca-3891195a03f2',
 '7769268b-9b01-48cf-8685-0053f7d5df06']

In [10]:
id2fname["fd741f4b-c011-4515-b2e3-c56a3af102a2"]

'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/internal/DefaultContext.kt'

### Список всех файлов

In [11]:
{fname for _, fname in id2fname.items()}

{'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/AnnotationExcluder.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/BaseRule.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/CodeSmell.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/Config.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/ConfigAware.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/ConfigProperty.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/ConfigValidator.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/ConsoleReport.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/DetektVisitor.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/Detektion.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/Entity.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/Extension.kt',
 'detekt-api/src/main/kotlin/io/gitlab/arturbosch/detekt/api/File

### Одна сессия выглядит так:

In [12]:
id2session["fd741f4b-c011-4515-b2e3-c56a3af102a2"]

{'_lookups': [{'prefix': '_',
   'suggestions': [{'text': 'DEFAULT_FLOAT_CONVERSION_FACTOR',
     'presentationText': 'DEFAULT_FLOAT_CONVERSION_FACTOR (io.gitlab.arturbosch.detekt.api): Int',
     'source': 'STANDARD',
     'createdLatency': 2444,
     'resultsetLatency': 2445,
     'indicatorLatency': 2445,
     'lookupLatency': 5634,
     'renderedLatency': 5750,
     'contributor': 'KotlinCompletionContributor',
     'contributorKind': 'kinds-descriptor-BasicCompletionSession.kt:403-LookupElementsCollector.kt:91',
     'kind': 'ANY'},
    {'text': 'DEFAULT_BUFFER_SIZE',
     'presentationText': 'DEFAULT_BUFFER_SIZE (kotlin.io): Int',
     'source': 'STANDARD',
     'createdLatency': 1484,
     'resultsetLatency': 1485,
     'indicatorLatency': 1487,
     'lookupLatency': 5495,
     'renderedLatency': 5750,
     'contributor': 'KotlinCompletionContributor',
     'contributorKind': 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91',
     'kind': 'ANY'},
   

### Готовим данные для тренировки, тестирования и валидации

In [13]:
xs, ys = utils.make_xy(id2feats, id2session)

In [14]:
len(xs)

3224

In [15]:
len(ys)

3224

### В xs контекстуальные признаки, в ys - части контрибьютра, в которых находился правильный вариант (и время работы этой части контрибьютора)

### Часть сессий пришлось выкинуть, так как для них не выполнялось некоторое допущение

In [16]:
xs[:10]

[{'ml_ctx_kotlin_plugin_version': '@snapshot@',
  'ml_ctx_kotlin_file_type': 'KT',
  'ml_ctx_cce_sessionUid': '3b6adb85-a76d-469d-82b1-6c54fd9ef373',
  'ml_ctx_common_line_num': '26.0',
  'ml_ctx_common_col_num': '40.0',
  'ml_ctx_common_indent_level': '2.0',
  'ml_ctx_common_is_in_line_beginning': '0',
  'ml_ctx_common_case_sensitivity': 'FIRST_LETTER',
  'ml_ctx_common_is_after_dot': '1',
  'ml_ctx_common_parent_1': 'KtDotQualifiedExpression',
  'ml_ctx_common_parent_2': 'KtProperty',
  'ml_ctx_common_parent_3': 'KtBlockExpression',
  'ml_ctx_common_parent_4': 'KtNamedFunction',
  'ml_ctx_common_parent_5': 'KtClassBody',
  'ml_ctx_common_parent_6': 'KtClass',
  'ml_ctx_common_parent_7': 'KtFile'},
 {'ml_ctx_kotlin_plugin_version': '@snapshot@',
  'ml_ctx_kotlin_file_type': 'KT',
  'ml_ctx_cce_sessionUid': '0aa5cd60-0142-4d11-8b8f-c2e508eb3613',
  'ml_ctx_common_line_num': '27.0',
  'ml_ctx_common_col_num': '13.0',
  'ml_ctx_common_indent_level': '2.0',
  'ml_ctx_common_is_in_line_beg

In [17]:
[
    (ind, item)
    for ind, item in enumerate(xs)
    if item["ml_ctx_cce_sessionUid"] == "fd741f4b-c011-4515-b2e3-c56a3af102a2"
]

[(2,
  {'ml_ctx_kotlin_plugin_version': '@snapshot@',
   'ml_ctx_kotlin_file_type': 'KT',
   'ml_ctx_cce_sessionUid': 'fd741f4b-c011-4515-b2e3-c56a3af102a2',
   'ml_ctx_common_line_num': '15.0',
   'ml_ctx_common_col_num': '17.0',
   'ml_ctx_common_indent_level': '2.0',
   'ml_ctx_common_is_in_line_beginning': '0',
   'ml_ctx_common_case_sensitivity': 'FIRST_LETTER',
   'ml_ctx_common_is_after_dot': '0',
   'ml_ctx_common_parent_1': 'KtPropertyAccessor',
   'ml_ctx_common_parent_2': 'KtProperty',
   'ml_ctx_common_parent_3': 'KtClassBody',
   'ml_ctx_common_parent_4': 'KtClass',
   'ml_ctx_common_parent_5': 'KtFile'})]

In [18]:
ys[:10]

[{(26,
   ('KotlinCompletionContributor',
    'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91'))},
 {(40,
   ('KotlinCompletionContributor',
    'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91'))},
 set(),
 {(708,
   ('KotlinCompletionContributor',
    'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91'))},
 {(104,
   ('KotlinCompletionContributor',
    'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91'))},
 {(147,
   ('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:711'))},
 {(59,
   ('KotlinCompletionContributor',
    'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91'))},
 {(51,
   ('KotlinCompletionContributor',
    'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91'))},
 {(44,
   ('KotlinCompletionContributor',
    'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91'))},
 {(

In [19]:
ys[2]

set()

### Пустое множество означает, что правильный вариант не сгенерировался ни в одной части

### Делаем pd.DataFrame с нужными данными и делаем метки (предсказываем часть контрибьютора, в которой есть правильный вариант и которая длится меньше всех):

In [20]:
%%time

data, labels = utils.make_data(xs, ys)

CPU times: user 23 ms, sys: 1.59 ms, total: 24.6 ms
Wall time: 23.7 ms


In [21]:
data.shape, labels.shape

((3022, 10), (3022, 1))

In [22]:
data.head()

,ml_ctx_kotlin_file_type,ml_ctx_common_is_in_line_beginning,ml_ctx_common_case_sensitivity,ml_ctx_common_is_after_dot,ml_ctx_common_parent_1,ml_ctx_common_parent_2,ml_ctx_common_parent_3,ml_ctx_common_line_num,ml_ctx_common_col_num,ml_ctx_common_indent_level
ml_ctx_cce_sessionUid,,,,,,,,,,
3b6adb85-a76d-469d-82b1-6c54fd9ef373,KT,0,FIRST_LETTER,1,KtDotQualifiedExpression,KtProperty,KtBlockExpression,26.0,40.0,2.0
0aa5cd60-0142-4d11-8b8f-c2e508eb3613,KT,0,FIRST_LETTER,0,KtContainerNode,KtIfExpression,KtBlockExpression,27.0,13.0,2.0
178b3f9f-5ca1-4349-9e94-88df79083635,KT,0,FIRST_LETTER,0,KtBlockExpression,KtContainerNodeForControlStructureBody,KtIfExpression,28.0,13.0,3.0
882310c1-90a8-4329-bcde-c684a8012e2d,KT,0,FIRST_LETTER,0,KtProperty,KtClassBody,KtClass,17.0,51.0,1.0
6c5510db-79c7-442c-97eb-d97852c3daf5,KT,0,FIRST_LETTER,0,KtBlockExpression,KtNamedFunction,KtClassBody,26.0,9.0,2.0


In [23]:
labels.head().style.set_properties(subset=['target'], **{'width': '700px'})

,target
session_id,
3b6adb85-a76d-469d-82b1-6c54fd9ef373,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
0aa5cd60-0142-4d11-8b8f-c2e508eb3613,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
178b3f9f-5ca1-4349-9e94-88df79083635,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
882310c1-90a8-4329-bcde-c684a8012e2d,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
6c5510db-79c7-442c-97eb-d97852c3daf5,"('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:711')"


### Разбиваем на train/dev/test

In [24]:
train, dev, test = utils.split(data, labels, id2fname, seed=12345)

In [25]:
train[0].shape, train[1].shape

((1835, 10), (1835, 1))

In [26]:
dev[0].shape, dev[1].shape

((639, 10), (639, 1))

In [27]:
test[0].shape, test[1].shape

((548, 10), (548, 1))

In [28]:
train[0].head()

,ml_ctx_kotlin_file_type,ml_ctx_common_is_in_line_beginning,ml_ctx_common_case_sensitivity,ml_ctx_common_is_after_dot,ml_ctx_common_parent_1,ml_ctx_common_parent_2,ml_ctx_common_parent_3,ml_ctx_common_line_num,ml_ctx_common_col_num,ml_ctx_common_indent_level
ml_ctx_cce_sessionUid,,,,,,,,,,
35d285fd-fd80-4662-9569-5f6afb4b4927,KT,0,FIRST_LETTER,0,KtParameter,KtParameterList,KtPrimaryConstructor,17.0,45.0,1.0
7c5c956a-e8e8-4e39-9e5a-9bf4b3a885d6,KT,0,FIRST_LETTER,0,KtValueArgument,KtValueArgumentList,KtCallExpression,15.0,43.0,2.0
244842ca-d590-4131-b519-850357903fb9,KT,0,FIRST_LETTER,0,KtValueArgument,KtValueArgumentList,KtConstructorDelegationCall,25.0,9.0,2.0
0a6378e0-3f87-4056-a208-9545efa7ea80,KT,0,FIRST_LETTER,0,KtClassBody,KtClass,KtFile,9.0,14.0,1.0
4d8187ce-8fb4-4a4a-b080-04b5a15ad00a,KT,0,FIRST_LETTER,0,KtBlockExpression,KtFunctionLiteral,KtLambdaExpression,46.0,41.0,3.0


In [29]:
train[1].head().style.set_properties(subset=['target'], **{'width': '700px'})

,target
session_id,
35d285fd-fd80-4662-9569-5f6afb4b4927,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
7c5c956a-e8e8-4e39-9e5a-9bf4b3a885d6,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
244842ca-d590-4131-b519-850357903fb9,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
0a6378e0-3f87-4056-a208-9545efa7ea80,"('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:711')"
4d8187ce-8fb4-4a4a-b080-04b5a15ad00a,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"


### Категориальные признаки - первые 7 штук

In [30]:
cat_features = list(range(7))

## Бейзлайн

### Находим самую частотную часть контрибьютора

In [31]:
train[1].value_counts("target")

target
('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')    1329
('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:711')                                               306
('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:636')                                                46
('KotlinCompletionContributor', 'kinds-BasicCompletionSession.kt:682')                                               36
('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:696')                                                28
('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:445')                                                19
('KotlinCompletionContributor', 'kinds-BasicCompletionSession.kt:656')                                               19
('KotlinCompletionContributor', 'kinds-BasicCompletionSession.kt:228')                                               12
('LiveTemplateCompletionContribut

In [32]:
train[1].value_counts("target")[0]

1329

In [33]:
train[1].value_counts("target").index[0]

"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"

### dev

### Предсказываем для всех сессий самую частотную часть

In [34]:
id2preds = {
    session_id : "('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
    for session_id in dev[1].index
}

In [35]:
utils.calc_metrics(id2preds, id2session)

{'latency': 382.04851330203445, 'recall': 0.7230046948356808}

### Считаем метрики текущей стратегии (которая в IDE) автодополнения (сейчас используется 100мс)

In [36]:
utils.print_current_metrics(id2preds, id2session)

approx recall (50ms): 0.844 (539 / 639)
approx recall (100ms): 0.890 (569 / 639)
approx recall (150ms): 0.900 (575 / 639)
approx recall (200ms): 0.906 (579 / 639)
mean approx latency (50ms): 340.188 (217380.000 / 639, skipped = 0)
mean approx latency (100ms): 390.188 (249330.000 / 639, skipped = 0)
mean approx latency (150ms): 440.188 (281280.000 / 639, skipped = 0)
mean approx latency (200ms): 490.188 (313230.000 / 639, skipped = 0)


### Аналогично для test

In [37]:
id2preds = {
    session_id : "('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
    for session_id in test[1].index
}

In [38]:
utils.calc_metrics(id2preds, id2session)

{'latency': 405.17518248175185, 'recall': 0.7427007299270073}

In [39]:
utils.print_current_metrics(id2preds, id2session)

approx recall (50ms): 0.788 (432 / 548)
approx recall (100ms): 0.827 (453 / 548)
approx recall (150ms): 0.845 (463 / 548)
approx recall (200ms): 0.854 (468 / 548)
mean approx latency (50ms): 316.464 (173422.000 / 548, skipped = 0)
mean approx latency (100ms): 366.464 (200822.000 / 548, skipped = 0)
mean approx latency (150ms): 416.464 (228222.000 / 548, skipped = 0)
mean approx latency (200ms): 466.464 (255622.000 / 548, skipped = 0)


### Числа на 16-м слайде про бейзлайн и текущий подход взяты отсюда ^

## Учим катбустовую модель

### Делаем катбустовые датасеты

In [40]:
train_dataset = cb.Pool(data=train[0],
                       label=train[1],
                       cat_features=cat_features)

dev_dataset = cb.Pool(data=dev[0],
                       label=dev[1],
                       cat_features=cat_features)

test_dataset = cb.Pool(data=test[0], label=test[1], cat_features=cat_features)

### Учим простую модель

In [41]:
model = cb.CatBoostClassifier(iterations=100,
                              learning_rate=1,
                              depth=2,
                              loss_function='MultiClass')
model.fit(train_dataset, verbose=0)

### Считаем accuracy

In [42]:
dev_preds = model.predict(dev_dataset)

In [43]:
(dev_preds == dev[1]).mean()

target    0.813772
dtype: float64

In [44]:
(test[1] == model.predict(test_dataset)).mean()

target    0.790146
dtype: float64

In [45]:
dev[1].value_counts("target")[0] / len(dev[1])

0.7104851330203443

### accuracy лучше, чем у бейзлайна, возможно, модель делает что-то полезное

### Теперь считаем уже recall и latency по сессиям:

In [46]:
id2preds = utils.make_preds(dev[1], dev_preds)

In [47]:
dev[1].head().style.set_properties(subset=['target'], **{'width': '700px'})

,target
session_id,
6b4e6b2c-5ea7-48a3-9e11-ebb3d0b72e81,"('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:696')"
cfb26963-0190-4fb0-9556-255949566a06,"('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:711')"
76746bca-bbbe-470d-ab6e-56ff8ebe0c80,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:866-LookupElementsCollector.kt:91')"
5808a373-9a6e-44e4-87bf-70c478b3d818,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"
49ce8072-356f-47a9-a656-f3e7cf337a54,"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"


In [48]:
dev_preds[:5]

array([["('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:711')"],
       ["('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:711')"],
       ["('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"],
       ["('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"],
       ["('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"]],
      dtype=object)

In [49]:
id2preds["76746bca-bbbe-470d-ab6e-56ff8ebe0c80"]

"('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')"

In [50]:
utils.calc_metrics(id2preds, id2session)

{'latency': 353.264475743349, 'recall': 0.8309859154929577}

### Натренированная модель проигрывает у текущей стратегии с 100мс по recall(recall = 0.89, latency = 390мс на dev)

### А если у текущей стратегии выставить 50мс (recall = 0.84, latency 340мс на dev), то модель проигрывает и по latency, и по recall

## Пробуем Random search

In [51]:
%%time
rng = random.Random(42)
history = []
for i in tqdm.tqdm(range(50)):
    model = cb.CatBoostClassifier(
        iterations=rng.choice([10, 50, 100, 200]),
        learning_rate=rng.choice([None, 1e-3, 1e-2, 1e-1, 3e-3, 1, 3, 10]),
        depth=rng.choice(list(range(4, 11))),
        l2_leaf_reg=rng.choice([None, 0.1, 1, 10]),
        random_strength=rng.choice([0.3, 1, 3]),
        loss_function='MultiClass',
        custom_metric='Accuracy',
    )
    model.fit(train_dataset, verbose=0)
    
    dev_metrics = utils.calc_metrics(utils.make_preds(dev[1], model.predict(dev_dataset)), id2session)
    history.append((i, dev_metrics, model))

  4%|▍         | 2/50 [00:02<01:02,  1.31s/it]learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
  8%|▊         | 4/50 [00:04<00:54,  1.19s/it]learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
 26%|██▌       | 13/50 [00:42<01:58,  3.21s/it]learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. You probably need to decrease learning rate.
 32%|███▏      | 16/50 [00:57<02:07,  3.76s/it]learning rate is greater than 1. You probably need to decrease learning rate.
learning rate is greater than 1. Y

CPU times: user 22min 28s, sys: 3min 39s, total: 26min 7s
Wall time: 3min 36s


### Сортируем по recall

In [52]:
print(
    "\n".join([
        f"model #{ind:02d}: latency = {metrics['latency']:.3f}, recall = {metrics['recall']:.4f}"
        for ind, metrics, _ in sorted(history, key=lambda x: (x[1]['recall'], x[0]), reverse=True)
    ])
)

model #37: latency = 369.124, recall = 0.9014
model #05: latency = 369.124, recall = 0.9014
model #03: latency = 369.124, recall = 0.9014
model #36: latency = 375.424, recall = 0.8951
model #29: latency = 363.296, recall = 0.8936
model #46: latency = 373.898, recall = 0.8920
model #45: latency = 374.156, recall = 0.8905
model #38: latency = 366.660, recall = 0.8873
model #19: latency = 371.061, recall = 0.8842
model #17: latency = 364.632, recall = 0.8842
model #20: latency = 371.016, recall = 0.8795
model #33: latency = 382.615, recall = 0.8685
model #28: latency = 382.615, recall = 0.8685
model #01: latency = 382.615, recall = 0.8685
model #25: latency = 387.061, recall = 0.8638
model #24: latency = 387.061, recall = 0.8638
model #23: latency = 386.944, recall = 0.8638
model #18: latency = 387.061, recall = 0.8638
model #09: latency = 387.061, recall = 0.8638
model #04: latency = 351.252, recall = 0.8623
model #27: latency = 365.052, recall = 0.8560
model #13: latency = 353.995, reca

### Считаем метрики на тестовой выборке для самой качественной модели

In [53]:
i, score, model = history[37]

In [54]:
utils.calc_metrics(utils.make_preds(test[1], model.predict(test_dataset)), id2session)

{'latency': 400.9215328467153, 'recall': 0.8594890510948905}

### Числа на 16-м слайде про качественную модель взяты отсюда ^

### Сортируем по latency

In [55]:
print(
    "\n".join([
        f"model #{ind:02d}: latency = {metrics['latency']:.3f}, recall = {metrics['recall']:.4f}"
        for ind, metrics, _ in sorted(history, key=lambda x: (x[1]['latency'], x[0]))
    ])
)

model #16: latency = 314.537, recall = 0.7919
model #06: latency = 318.213, recall = 0.7590
model #49: latency = 320.258, recall = 0.8028
model #10: latency = 328.890, recall = 0.7950
model #47: latency = 335.563, recall = 0.7778
model #02: latency = 341.851, recall = 0.8013
model #08: latency = 345.322, recall = 0.8372
model #04: latency = 351.252, recall = 0.8623
model #13: latency = 353.995, recall = 0.8529
model #39: latency = 354.163, recall = 0.7966
model #34: latency = 354.468, recall = 0.7966
model #43: latency = 354.729, recall = 0.8435
model #40: latency = 355.319, recall = 0.7981
model #32: latency = 355.451, recall = 0.7872
model #26: latency = 355.457, recall = 0.7950
model #07: latency = 355.527, recall = 0.7966
model #15: latency = 357.715, recall = 0.7950
model #14: latency = 358.531, recall = 0.8232
model #48: latency = 358.674, recall = 0.8044
model #11: latency = 359.177, recall = 0.8044
model #22: latency = 360.743, recall = 0.7997
model #41: latency = 361.573, reca

### Считаем метрики на тестовой выборке для самой быстрой модели

In [56]:
i, score, model = history[16]
utils.calc_metrics(utils.make_preds(test[1], model.predict(test_dataset)), id2session)

{'latency': 362.7609489051095, 'recall': 0.7645985401459854}

### Числа на слайде #16 про быструю модель взяты отсюда ^

## Метрики на 15-м слайде были посчитаны так

In [57]:
%%time


!python3 script.py \
    "/Users/ifled/projects/csc/ide_perf_practice/detekt/completion-evaluation/2022-05-18_16-07-33/data/files/jsons" \
    | tail -100

['script.py', '/Users/ifled/projects/csc/ide_perf_practice/detekt/completion-evaluation/2022-05-18_16-07-33/data/files/jsons']
    2464.600 (73938 / 30) - ('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:438')
    1295.800 (38874 / 30) - ('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:445')
    751.083 (9013 / 12) - ('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:636')
    586.234 (27553 / 47) - ('KotlinCompletionContributor', 'kinds-descriptor-BasicCompletionSession.kt:859-LookupElementsCollector.kt:91')
    423.500 (847 / 2) - ('KotlinCompletionContributor', 'kinds-BasicCompletionSession.kt:228')
    233.500 (1868 / 8) - ('KotlinCompletionContributor', 'kind-BasicCompletionSession.kt:287')
    139.250 (557 / 4) - ('KotlinCompletionContributor', 'kind-StaticMembersCompletion.kt:146')
    110.500 (1547 / 14) - ('LiveTemplateCompletionContributor', '=emptyContributorKind=')
    52.500 (420 / 8) - ('KotlinCompletionContributor', 'kind-BasicC

### Задержка текущего автодополнения - это `mean approx latency (100ms)`
### Recall текущего автодополнения - это `contiguous approx recall (100ms)`
### Задержка идеального переупорядочивания - это `min reorder oracle latency`
### Recall идеального переупорядочивания - это `contiguous recall`